In [1]:
import numpy as np
import nltk
from nltk.text import TextCollection
from pymongo import MongoClient
from sklearn.feature_extraction.text import TfidfVectorizer
import pandas as pd
from numpy import dot
from numpy.linalg import norm
import Preprocessor
from bson import ObjectId
import operator

In [2]:
client = MongoClient("localhost")
db=client.crawlerdb_WORK_TFIDF_2
collection = db.crawlerdb_lower_withSW
result_documents_list_REAL = list(collection.find({})) # work-data-collection
print("We got ", len(result_documents_list_REAL), " Documents")

We got  23733  Documents


In [4]:
#result_document_text_list = [a["text"] for a in result_documents_list_REAL] # wir müssen erst result_documents_list["text"] "entpacken" von cursor
result_document_text_list = list()
for i in range(len(result_documents_list_REAL)):
    try:
        result_document_text_list.append(result_documents_list_REAL[i]["text"]) # Fehler: KeyError: muss abgefangen werden
    except KeyError:
        pass
print("We still have ", len(result_document_text_list), " Documents")

We still have  23733  Documents


In [5]:
tfidf = TfidfVectorizer()
tfidf_values = tfidf.fit_transform(result_document_text_list) # output= <class 'scipy.sparse.csr.csr_matrix'>

In [6]:
type(tfidf_values)

scipy.sparse.csr.csr_matrix

In [7]:
result_dict_with_ids = {}
for i in range(len(result_documents_list_REAL)):
    key = str(result_documents_list_REAL[i]["_id"]) # 5c2fb3c988df422822370767
    result_dict_with_ids[key] = tfidf_values[i]

In [8]:
result_dict_with_ids["5c2fb3c988df422822370767"]

<1x45374 sparse matrix of type '<class 'numpy.float64'>'
	with 64 stored elements in Compressed Sparse Row format>

### finished pre TF_IDF - Calculation here

--------------------------------

In [9]:
search_term = "zwei maskierte männer haben heute"

preprocessor = Preprocessor.Preprocessor()
tokenized_words = preprocessor.tokenizing_without_punc(search_term)
stemmed_search_term = preprocessor.stemming_words(tokenized_words)
stemmed_search_term

['zwei', 'maskiert', 'maenn', 'hab', 'heut']

In [10]:
print(tokenized_words)
result_ids = set()
collection_inverse_index = db.crawlerdb_INVERSE_INDEX
for i in stemmed_search_term:
    print(i)
    record = collection_inverse_index.find({"word": i})
    ids_in_tupel_now_in_list = list() # ["sadfasdf", "sadfsda", ...]
    for m in record:
        current = list(m["documents"]) # ["asdfsadf"]
        for a in current:
            result_ids.add(a)
type(result_ids)

['zwei', 'maskierte', 'maenner', 'haben', 'heute']
zwei
maskiert
maenn
hab
heut


set

### match ids to document

In [11]:
object_ids = list()
for i in result_ids:
    object_ids.append(ObjectId(i))

In [12]:
less_document_actually = list()
for i in result_ids:
    less_document_actually.append(result_dict_with_ids[i])

### Get from our TF-IDF - Vector the given important documents

### calculate tf-idf

In [13]:
tfidf_search_term = tfidf.transform([search_term]) # transform passt idf nicht an
print(tfidf_search_term.todense())

[[0. 0. 0. ... 0. 0. 0.]]


In [14]:
less_documents = list()
for a in range(len(less_document_actually)):
    less_documents.append(less_document_actually[a].toarray().reshape(-1))
print("We have the following number of important selected documents for example from inverse index: ", len(less_documents))
print("We have ",less_documents[0].size, " diffrent words")

cos_sim_results = list()
for i in range(len(less_documents)): # actually all values in tfidf_values.todense().shape[0] but that is not possible
        #aufpassen: search term was array now its a string
    tfidf_values_tmp = less_documents[i]
    tfidf_search_term_tmp = np.array(tfidf_search_term.todense()).reshape(-1)
    
    first = np.dot(tfidf_values_tmp, tfidf_search_term_tmp)
    second = (norm(tfidf_values_tmp)*norm(tfidf_search_term_tmp))
    cos_sim = first * 1.0/second
    cos_sim_results.append(cos_sim)
    #print("cos sim of the current - document with the search-string is: ",cos_sim)

We have the following number of important selected documents for example from inverse index:  6759
We have  45374  diffrent words


In [15]:
new_ids_with_scores = {}
i = 0
for a in result_ids:
    new_ids_with_scores[a] = cos_sim_results[i]
    i+=1

In [16]:
sorted_d = sorted(new_ids_with_scores.items(), key=operator.itemgetter(1), reverse=True)[0:20]
sorted_d

[('5c2fb3f388df422822371826', 0.34183384067230027),
 ('5c2fb3ce88df42282237098a', 0.29872748793568155),
 ('5c2fb3d388df422822370b3f', 0.283940111649091),
 ('5c2fb40b88df42282237219d', 0.2740493354172982),
 ('5c2fb3e288df422822371167', 0.24469651063750925),
 ('5c2fb40088df422822371d58', 0.23269913382686297),
 ('5c2fb41e88df422822372901', 0.23015837407963186),
 ('5c2fb3de88df422822370fdb', 0.2298265510043141),
 ('5c2fb3c988df422822370767', 0.2191374831768901),
 ('5c2fe74088df4228e78630b7', 0.21758569944231765),
 ('5c2fb3d688df422822370cc4', 0.21080060687491256),
 ('5c2fb41188df4228223723f7', 0.20322017904480022),
 ('5c2fb3da88df422822370e26', 0.20121357613811386),
 ('5c2fb40c88df422822372212', 0.1992939653297332),
 ('5c2fb41888df4228223726ca', 0.1958831997975754),
 ('5c2fb3d788df422822370d19', 0.19513370354985532),
 ('5c2fb3c988df42282237076a', 0.1895255063239579),
 ('5c2fb40788df422822371fe2', 0.18892710736724608),
 ('5c2fb40f88df42282237234f', 0.1866542661527229),
 ('5c2fb3f988df422822

In [17]:
ids = [ObjectId(sorted_d[i][0]) for i in range(len(sorted_d))]

In [18]:
documents = collection.find({"_id" : {"$in" : ids}})
print(type(documents[0]))
documents[0]

<class 'dict'>


{'_id': ObjectId('5c2fb3c988df422822370767'),
 'date': '23.11.2018',
 'title': 'duo überfällt apotheke',
 'location': 'Neukölln',
 'text': 'zwei maskierte männer haben heute vormittag eine apotheke in britz überfallen bisherigen erkenntnissen zufolge betrat das duo gegen 1130 uhr das geschäft am britzer damm ein täter ging mit einem messer in der hand zu einer 35 jahre alten angestellten die gerade eine kundin bediente bedrohte sie und verlangte geld nachdem die mitarbeiterin die kasse geöffnet hatte griffen die räuber selbst in diese und flüchteten anschließend in richtung hannemannstraße das raubkommissariat der polizeidirektion 5 übernahm die ermittlungen',
 'number': ['Nr. 2455', '']}

In [19]:
list_of_documents = list(documents)
for i in range(len(list_of_documents)):
    list_of_documents[i]["cossim"] = sorted_d[i][1]

[{'_id': ObjectId('5c2fb3c988df422822370767'),
  'date': '23.11.2018',
  'title': 'duo überfällt apotheke',
  'location': 'Neukölln',
  'text': 'zwei maskierte männer haben heute vormittag eine apotheke in britz überfallen bisherigen erkenntnissen zufolge betrat das duo gegen 1130 uhr das geschäft am britzer damm ein täter ging mit einem messer in der hand zu einer 35 jahre alten angestellten die gerade eine kundin bediente bedrohte sie und verlangte geld nachdem die mitarbeiterin die kasse geöffnet hatte griffen die räuber selbst in diese und flüchteten anschließend in richtung hannemannstraße das raubkommissariat der polizeidirektion 5 übernahm die ermittlungen',
  'number': ['Nr. 2455', ''],
  'cossim': 0.34183384067230027},
 {'_id': ObjectId('5c2fb3c988df42282237076a'),
  'date': '24.11.2018',
  'title': 'bekleidungsgeschäft überfallen und beraubt',
  'location': 'Charlottenburg-Wilmersdorf',
  'text': 'maskierte überfielen gestern abend ein geschäft in charlottenburg gegen 2120 uh